<a href="https://colab.research.google.com/github/arsendoinychko/Cp-decomposition/blob/main/res.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time

# Гіперпараметри
batch_size = 64
learning_rate = 0.001
num_epochs = 2

# Завантаження та нормалізація набору даних CIFAR-10
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Завантаження попередньо навченого VGG-16 і модифікація для 10 класів CIFAR-10
class ModifiedVGG16(nn.Module):
    def __init__(self):
        super(ModifiedVGG16, self).__init__()
        self.vgg16 = torchvision.models.vgg16(pretrained=True)
        self.vgg16.classifier[6] = nn.Linear(4096, 10)  # Змінюємо останній повнозв'язний шар на 10 виходів

    def forward(self, x):
        x = self.vgg16(x)
        return x

# Ініціалізація моделі, функції втрат та оптимізатора
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ModifiedVGG16()

# Переводимо модель на декілька GPU, якщо доступні
if torch.cuda.device_count() > 1:
    print(f'Using {torch.cuda.device_count()} GPUs!')
    model = nn.DataParallel(model)

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Навчання моделі
for epoch in range(num_epochs):
    start_time = time.time()  # Початок відліку часу
    running_loss = 0.0
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # Кожні 100 міні-батчів
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0
    epoch_duration = time.time() - start_time  # Кінець відліку часу
    print(f'Epoch [{epoch + 1}/{num_epochs}] completed in {epoch_duration:.2f} seconds')

print('Finished Training')

# Тестування моделі
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

# Збереження моделі
torch.save(model.state_dict(), 'vgg16_cifar10_model.pth')

In [ ]:
# Тестування моделі
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

# Збереження моделі
torch.save(model.state_dict(), 'vgg16_cifar10_model.pth')

In [ ]:
print(model)

In [ ]:
if isinstance(model, nn.DataParallel):
    model = model.module

# Отримуємо третій згортковий шар
conv_layer1 = model.vgg16.features[0]
conv_layer2 = model.vgg16.features[2]
conv_layer3 = model.vgg16.features[5]  # Індекс 4 відповідає третьому згортковому шару (починаючи з нуля)
conv_layer4 = model.vgg16.features[7]
conv_layer5 = model.vgg16.features[10]
conv_layer6 = model.vgg16.features[12]
conv_layer7 = model.vgg16.features[14]
conv_layer8 = model.vgg16.features[17]
conv_layer9 = model.vgg16.features[19]
conv_layer10 = model.vgg16.features[21]
conv_layer11 = model.vgg16.features[24]
conv_layer12 = model.vgg16.features[26]
conv_layer13 = model.vgg16.features[28]

print(conv_layer7)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tensorly.decomposition import parafac
from tensorly.decomposition import parafac
import tensorly as tl

def cp_decomposition_conv_layer(layer):
    #device = layer.weight.device

    tl.set_backend('pytorch')
    S = layer.in_channels
    T = layer.out_channels
    rank = S*T/(S+T)
    rank = round(rank)
    print(rank)

   # weights = layer.weight.data.to(device)
    weights,(last, first, vertical, horizontal) = parafac(layer.weight.data, rank=rank, init='svd')

    pointwise_s_to_r_layer = torch.nn.Conv2d(in_channels=first.shape[0], \
            out_channels=first.shape[1], kernel_size=1, stride=1, padding=0,
            dilation=layer.dilation, bias=False)

    depthwise_vertical_layer = torch.nn.Conv2d(in_channels=vertical.shape[1],
            out_channels=vertical.shape[1], kernel_size=(vertical.shape[0], 1),
            stride=1, padding=(layer.padding[0], 0), dilation=layer.dilation,
            groups=vertical.shape[1], bias=False)

    depthwise_horizontal_layer = \
        torch.nn.Conv2d(in_channels=horizontal.shape[1], \
            out_channels=horizontal.shape[1],
            kernel_size=(1, horizontal.shape[0]), stride=layer.stride,
            padding=(0, layer.padding[0]),
            dilation=layer.dilation, groups=horizontal.shape[1], bias=False)

    pointwise_r_to_t_layer = torch.nn.Conv2d(in_channels=last.shape[1], \
            out_channels=last.shape[0], kernel_size=1, stride=1,
            padding=0, dilation=layer.dilation, bias=True)

    pointwise_r_to_t_layer.bias.data = layer.bias.data

    depthwise_horizontal_layer.weight.data = \
        torch.transpose(horizontal, 1, 0).unsqueeze(1).unsqueeze(1)
    depthwise_vertical_layer.weight.data = \
        torch.transpose(vertical, 1, 0).unsqueeze(1).unsqueeze(-1)
    pointwise_s_to_r_layer.weight.data = \
        torch.transpose(first, 1, 0).unsqueeze(-1).unsqueeze(-1)
    pointwise_r_to_t_layer.weight.data = last.unsqueeze(-1).unsqueeze(-1)

    new_layers = [pointwise_s_to_r_layer, depthwise_vertical_layer, \
                    depthwise_horizontal_layer, pointwise_r_to_t_layer]

    return nn.Sequential(*new_layers)

In [ ]:
from tensorly.decomposition import parafac
tl.set_backend('pytorch')

S = conv_layer5.in_channels
T = conv_layer5.out_channels
rank = S*T/(S+T)
rank = round(rank)
print(rank)

   # weights = layer.weight.data.to(device)
weights,(last, first, vertical, horizontal) = parafac(conv_layer5.weight.data, rank=rank, init='svd')

In [ ]:
new_layers5 = cp_decomposition_conv_layer(conv_layer5.cpu())
new_layers6 = cp_decomposition_conv_layer(conv_layer6.cpu())
new_layers7 = cp_decomposition_conv_layer(conv_layer7.cpu())
new_layers8 = cp_decomposition_conv_layer(conv_layer8.cpu())

In [ ]:
model.vgg16.features[5] = nn.Sequential(*new_layer3).cuda()

In [ ]:
for epoch in range(num_epochs):
    start_time = time.time()  # Початок відліку часу
    running_loss = 0.0
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # Кожні 100 міні-батчів
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0
    epoch_duration = time.time() - start_time  # Кінець відліку часу
    print(f'Epoch [{epoch + 1}/{num_epochs}] completed in {epoch_duration:.2f} seconds')

print('Finished Training')

In [ ]:
# Тестування моделі
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

# Збереження моделі
torch.save(model.state_dict(), 'vgg16_cifar10_model.pth')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tensorly.decomposition import parafac
import tensorly as tl

# Визначення функції CP-декомпозиції
def torch_cp_decomp(layer, rank):
    W = layer.weight.data

    weights,(last, first, vertical, horizontal) = parafac(W, rank=rank, init='random')


    pointwise_s_to_r_layer = nn.Conv2d(in_channels=first.shape[0],
                                       out_channels=first.shape[1],
                                       kernel_size=1,
                                       padding=0,
                                       bias=False)

    depthwise_r_to_r_layer = nn.Conv2d(in_channels=rank,
                                       out_channels=rank,
                                       kernel_size=vertical.shape[0],
                                       stride=layer.stride,
                                       padding=layer.padding,
                                       dilation=layer.dilation,
                                       groups=rank,
                                       bias=False)

    pointwise_r_to_t_layer = nn.Conv2d(in_channels=last.shape[1],
                                       out_channels=last.shape[0],
                                       kernel_size=1,
                                       padding=0,
                                       bias=True)

    if layer.bias is not None:
        pointwise_r_to_t_layer.bias.data = layer.bias.data

    sr = first.t_().unsqueeze_(-1).unsqueeze_(-1)
    rt = last.unsqueeze_(-1).unsqueeze_(-1)
    rr = torch.stack([vertical.narrow(1, i, 1) @ torch.t(horizontal).narrow(0, i, 1) for i in range(rank)]).unsqueeze_(1)

    pointwise_s_to_r_layer.weight.data = sr
    pointwise_r_to_t_layer.weight.data = rt
    depthwise_r_to_r_layer.weight.data = rr

    new_layers = [pointwise_s_to_r_layer,
                  depthwise_r_to_r_layer, pointwise_r_to_t_layer]
    return new_layers